In [7]:
%env kubectl=/usr/bin/kubectl
%env istio=/Users/gsjurseth/istio-1.0.4
%env istioctl=/Users/gsjurseth/istio-1.0.4/bin/istioctl
%env ingress=35.225.233.232
%env demo=/Users/gsjurseth/dev/fazio/microservices-demo
%env apigee-istio=/Users/gsjurseth/istio/apigee-istio
%env mypass=`dapass`

env: kubectl=/usr/bin/kubectl
env: istio=/Users/gsjurseth/istio-1.0.4
env: istioctl=/Users/gsjurseth/istio-1.0.4/bin/istioctl
env: ingress=35.225.233.232
env: demo=/Users/gsjurseth/dev/fazio/microservices-demo
env: apigee-istio=/Users/gsjurseth/istio/apigee-istio
env: mypass=`dapass`


---
## Access the API

In [8]:
!curl $ingress/products

---
## Wire up Apigee

The following configures how Istio passes data to the Apigee adapter.

In [5]:
%%bash
cat <<EOF | $kubectl apply -f -

# instance configuration for template 'apigee.analytics'
apiVersion: config.istio.io/v1alpha2
kind: analytics
metadata:
  name: apigee
  namespace: istio-system
spec:
  api_key: request.api_key | request.headers["x-api-key"] | ""
  api_proxy: api.service | destination.service.host | ""
  response_status_code: response.code | 0
  client_ip: source.ip | ip("0.0.0.0")
  request_verb: request.method | ""
  request_uri: request.path | ""
  useragent: request.useragent | ""
  client_received_start_timestamp: request.time
  client_received_end_timestamp: request.time
  target_sent_start_timestamp: request.time
  target_sent_end_timestamp: request.time
  target_received_start_timestamp: response.time
  target_received_end_timestamp: response.time
  client_sent_start_timestamp: response.time
  client_sent_end_timestamp: response.time
  api_claims: # from jwt
    json_claims: request.auth.raw_claims | ""
---
# instance configuration for template 'apigee.authorization'
apiVersion: config.istio.io/v1alpha2
kind: authorization
metadata:
  name: apigee
  namespace: istio-system
spec:
  subject:
    user: ""
    groups: ""
    properties:
      api_key: request.api_key | request.headers["x-api-key"] | ""
      json_claims: request.auth.raw_claims | ""
  action:
    namespace: destination.namespace | "default"
    service: api.service | destination.service.host | ""
    path: api.operation | request.path | ""
    method: request.method | ""

EOF

analytics.config.istio.io/apigee unchanged
authorization.config.istio.io/apigee unchanged


---
## Analytics Rule

This lets Istio know that we want to send all Istio telemetry to Apigee via the adapter.

In [6]:
%%bash
cat <<EOF | $kubectl apply -f -

# Defines rules to apply the Apigee mixer adapter to requests.
# In the rule below, we apply Apigee authorization and analytics
# as defined in the apigee-handler (handler.yaml) to all intra-mesh
# requests.
---
apiVersion: config.istio.io/v1alpha2
kind: rule
metadata:
  name: apigee-rule
  namespace: istio-system
spec:
  match: context.reporter.kind == "inbound" && destination.namespace == "default"
  actions:
  - handler: apigee-handler.apigee.istio-system
    instances:
    - apigee.analytics

EOF

rule.config.istio.io/apigee-rule configured


## Analytics Reporting

Now we have analytics coming from our Istio services.

https://apigee.com/platform/emea-poc15/devices

---
## Apigee API Product

An API Product is how I will expose the API to developers.

https://apigee.com/platform/emea-poc15/products/Tracking%20Details

---
## Protect the API with an API Key

In [7]:
%%bash
cat <<EOF | $kubectl apply -f -

# Defines rules to apply the Apigee mixer adapter to requests.
# In the rule below, we apply Apigee authorization and analytics
# as defined in the apigee-handler (handler.yaml) to all intra-mesh
# requests.
---
apiVersion: config.istio.io/v1alpha2
kind: rule
metadata:
  name: apigee-rule
  namespace: istio-system
spec:
  match: context.reporter.kind == "inbound" && destination.service.name == "productcatalogservice"
  actions:
  - handler: apigee-handler.apigee.istio-system
    instances:
    - apigee.analytics
    - apigee.authorization
    - apigee.quota
    
EOF

rule.config.istio.io/apigee-rule configured


---
## Create an App to access the API Product

The Developer Portal lets our developers create their own Apps to access protected API Product.

https://emea-poc15-fazioapiportal.apigee.io/

--- 
## Now we have an API Key .. Let's try hitting the api one more time with the key

---
### Check the result

In [10]:
!curl -i http://$ingress/products/OLJCESPC7Z -H "x-api-key: TiZKngjgrJxHqCRblIGvQgaKSvZihfGR" 

HTTP/1.1 200 OK
content-type: application/json
x-envoy-upstream-service-time: 786
grpc-status: 0
grpc-message: 
content-length: 249
date: Mon, 03 Dec 2018 12:53:07 GMT
server: envoy

{"id":"OLJCESPC7Z","name":"Vintage Typewriter","description":"This typewriter looks good in your living room.","picture":"/static/img/products/typewriter.jpg","priceUsd":{"currencyCode":"USD","units":"67","nanos":990000000},"categories":["vintage"]}

---
## Rollback APIKey protection

In [11]:
%%bash
cat <<EOF | $kubectl apply -f -

# Defines rules to apply the Apigee mixer adapter to requests.
# In the rule below, we apply Apigee authorization and analytics
# as defined in the apigee-handler (handler.yaml) to all intra-mesh
# requests.
---
apiVersion: config.istio.io/v1alpha2
kind: rule
metadata:
  name: apigee-rule
  namespace: istio-system
spec:
  match: context.reporter.kind == "inbound" && destination.service.name == "productcatalogservice"
  actions:
  - handler: apigee-handler.apigee.istio-system
    instances:
    - apigee.analytics

EOF

rule.config.istio.io/apigee-rule configured


In [14]:
!curl -i http://$ingress/products/OLJCESPC7Z

HTTP/1.1 503 Service Unavailable
content-length: 57
content-type: text/plain
date: Mon, 03 Dec 2018 21:04:24 GMT
server: envoy

upstream connect error or disconnect/reset before headers